In [1]:
import cv2  #importing the necessary modules
import tensorflow as tf
from keras_preprocessing import image
import numpy as np

In [2]:
face_cascade = cv2.CascadeClassifier('C:/Users/vasil/Desktop/tensorflow/FER/data/haarcascade_frontalface_default.xml')
model =tf.keras.models.load_model('bestmodel') #load the model
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'} #setting the emotions

In [3]:
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, cap_image = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv2.cvtColor(cap_image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1,4)
    for (x,y,w,h) in faces:
        cv2.rectangle(cap_image, (x,y), (x+w, y+h), (0,255,0),2) #code to show rectangles and emotions in camera window
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        predictions = model.predict(img_pixels)
        emotion_label = np.argmax(predictions)
        emotion_prediction = label_dict[emotion_label]
        cv2.putText(cap_image, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,0), 1 )
        resize_image = cv2.resize(cap_image, (1000,700))
        
        
    # Display the resulting frame
    cv2.imshow('emotion', resize_image)
    if cv2.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()